# Анализ новостных сообщений

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Teaching/Netology/NLP/data/lenta-ru-news.csv', usecols = ['title', 'text', 'topic', 'tags'])

In [ ]:
df.tail()

,title,text,topic,tags
800970,Шнуров раскритиковал Гагарину на «Голосе»,Певец Сергей Шнуров раскритиковал свою коллегу...,NaN,ТВ и радио
800971,В России предложили изменить правила взыскания...,Министерство юстиции России предложило изменит...,NaN,Все
800972,В России назвали «черную дату» для Европы,Испытание США ранее запрещенной Договором о ли...,NaN,Политика
800973,Россиянам пообещали аномально теплую погоду,В ближайшие дни в европейской части России пог...,NaN,Общество
800974,В конкурсе прогнозов на АПЛ разыграют 100 тыся...,Ведущие футбольные чемпионаты ушли на зимние к...,NaN,Английский футбол


In [ ]:
df['topic'].value_counts()

Россия               160445
Мир                  136621
Экономика             79528
Спорт                 64413
Культура              53797
Бывший СССР           53402
Наука и техника       53136
Интернет и СМИ        44663
Из жизни              27605
Дом                   21734
Силовые структуры     19596
Ценности               7766
Бизнес                 7399
Путешествия            6408
69-я параллель         1268
Крым                    666
Культпросвет            340
Легпром                 114
Библиотека               65
Оружие                    3
ЧМ-2014                   2
Сочи                      1
МедНовости                1
Name: topic, dtype: int64

In [ ]:
df['tags'].value_counts()

Все               453762
Политика           40716
Общество           35202
Украина            22523
Происшествия       19825
                   ...  
Нацпроекты             6
Мировой опыт           6
Вооружение             3
69-я параллель         1
Инновации              1
Name: tags, Length: 94, dtype: int64

In [ ]:
sample = df[df['tags'] == 'Кино']
print(len(sample))

10720


### Предобработка

#### Оставляем только слова:

In [ ]:
import re
regex = re.compile("[А-Яа-я]+")

In [ ]:
def words_only(text, regex=regex):
    return " ".join(regex.findall(text))


sample.text = sample.text.str.lower()
sample.text = sample.text.apply(words_only)

In [ ]:
sample.text.iloc[0]

'американского актера уэсли снайпса в четверг приговорили к трем годам тюремного заключения за неуплату налогов в период с по годы передает агентство таким образом актер получил максимальный срок за каждый из трех эпизодов неуплаты налогов ранее со снайпса сняли более серьезные обвинения в мошенничестве и сговоре с целью ввести в заблуждение налоговую службу на судебном заседании в четверг адвокаты исполнителя роли блейда в одноименной трилогии представили три десятка писем от членов его семьи друзей и коллег актеров в которых описывался его прекрасный характер защита настаивала что снайпсу будет достаточно условного наказания так как ранее он к суду не привлекался обвинение в неуплате налогов на сумму в миллиона долларов было предъявлено снайпсу и еще двум соответчикам по делу в году как утверждали адвокаты актера он не осознавал что действует незаконно между тем приговоры в отношении соответчиков снайпса эдди рея кана и дугласа росайла будут вынесены в ближайшее время по версии обвин

#### Убираем стоп-слова:

In [ ]:
from nltk.corpus import stopwords
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д']

print(mystopwords)

LookupError: ignored

In [ ]:
def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""
sample.text = sample.text.apply(remove_stopwords)   

In [ ]:
sample.text.iloc[0][:100]

#### Лемматизируем

In [ ]:
from pymystem3 import Mystem

In [ ]:
m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

In [ ]:
%%time 
sample.text = sample.text.apply(lemmatize)

In [ ]:
sample.text.iloc[0][:100]

#### Удаление стоп-лемм

In [ ]:
mystoplemmas = ['который', 'прошлый', 'сей', 'свой', 'наш', 'мочь', 'год']
def  remove_stoplemmas(text, mystoplemmas = mystoplemmas):
    try:
        return " ".join([token for token in text.split() if not token in mystoplemmas])
    except:
        return ""

sample.text = sample.text.apply(remove_stoplemmas)   

### Частотный словарь и облако слов

In [ ]:
!pip3 install wordcloud

In [ ]:
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
lemmata = [lemma for text in sample.text for lemma in text.split()]

In [ ]:
cnt = Counter(lemmata)

In [ ]:
for i in cnt.most_common(10):
    print(i)

In [ ]:
word_freq = [i for i in cnt.most_common(100)]
wd = WordCloud(background_color = 'white')
wd.generate_from_frequencies(dict(word_freq))
plt.figure()
plt.imshow(wd, interpolation = 'bilinear')
plt.axis('off')
plt.show()

## Извлечение ключевых словосочетаний


Ключевые слова и словосочетания сложно определить формально. Поскольку определений ключевых слов и словосочетаний множество, существует масса методов их извлечения:
* с учителем VS без учителя
* по частотам VS посложнее
* из одного текста VS из коллекции текстов
* слова (униграммы) VS биграммы VS $N$-граммы
* термины VS именованные сущности VS коллокации
* последовательные слова VS с использованием окна

### Основные этапы извлечения ключевых слов и словосочетаний:
1. Порождение кандидатов
2. Оценка свойст кандидатов
3. Выбор лучших кандидатов

### Основные методы извлечения ключевых слов и словосочетаний:
* Морфологические шаблоны
* Меры ассоциации биграмм: PMI, T-Score, LLR
* Графовые методы: TextRank [Mihalcea, Tarau, 2004]
* Синтаксические шаблоны


### Морфологические шаблоны

Можно использовать парсер  Yargy. 

Простейший шаблон ПРИЛ + СУЩ

```
S -> Adj<gnc-agr[1]> Noun<rt,gnc-agr[1]>; 
```

### Использование мер связности 


$w_1, w_2$ − два слова

$f(w_1), f(w_2)$ − их частоты

$f(w_1, w_2)$ − совместная частота биграммы $w_1 w_2$

$N$ − число слов

$PMI(w_1, w_2) = \log \frac{f(w_1, w_2)}{f(w_1)f(w_2)}$

$T-score(w_1, w_2) = \frac{f(w_1,w_2)-f(w_1)*f(w_2)}{f(w_1,w_2)/N}$

Другие меры связности: $\chi^2$

![chi-square](chi-square-formula.jpg)

### На практике

Получаем из датафрейма списки по разных топиков:

In [ ]:
def get_topic_to_tokens(df):
    tokens_by_topic = {}
    for topic in set(df['topic'].dropna()):
        
        # берём только относительно большие темы
#         print('---')
#         print(df['topic'])
        if df['topic'].dropna().value_counts()[topic] > 100:
            
            # берём по сто случайных текстов из каждой темы
            sample = df[df['topic']==topic].sample(n=100)
            
            # предобрабатываем
            sample.text = sample.text.str.lower()
            sample.text = sample.text.apply(words_only)
            sample.text = sample.text.apply(remove_stopwords) 
            sample.text = sample.text.apply(lemmatize)
            sample.text = sample.text.apply(remove_stoplemmas) 

            tokens_by_topic[topic] = [tok for text in sample.text for tok in text.split()]
            
    return tokens_by_topic

In [ ]:
tokens_by_topic = get_topic_to_tokens(df)

In [ ]:
tokens_by_topic.keys()

Выберем тему, из текстов про которую будем извлекать ключевые слова:

In [ ]:
topic_texts = tokens_by_topic['Путешествия']

In [ ]:
topic_texts[:10]

Извлекаем биграммы по разным мерам связности:

In [ ]:
import nltk
from nltk.collocations import *

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures() # класс для мер ассоциации биграм
finder = BigramCollocationFinder.from_words(topic_texts) # класс для хранения и извлечения биграм

In [ ]:
N_best = 100 # число извлекаемых биграм

In [ ]:
%%time 
finder.apply_freq_filter(3) # избавимся от биграм, которые встречаются реже трех раз

# выбираем топ-100 биграм по каждой мере
raw_freq_ranking = [' '.join(i) for i in finder.nbest(bigram_measures.raw_freq, N_best)]
tscore_ranking = [' '.join(i) for i in finder.nbest(bigram_measures.student_t, N_best)]
pmi_ranking =  [' '.join(i) for i in finder.nbest(bigram_measures.pmi, N_best)]
chi2_ranking =  [' '.join(i) for i in finder.nbest(bigram_measures.chi_sq, N_best)]

Результаты:

In [ ]:
rankings = pd.DataFrame({
    'chi2': chi2_ranking,
    't-score' : tscore_ranking,
    'pmi': pmi_ranking,
    'raw_freq':raw_freq_ranking
})
rankings = rankings[['raw_freq', 'pmi', 't-score', 'chi2']]
rankings.head(10)

Похожи ли списки биграм? Давайте посчитаем корреляцию и визуализируем.

In [ ]:
from scipy.stats import spearmanr
import seaborn as sns
%matplotlib inline

In [ ]:
corr = spearmanr(rankings).correlation

In [ ]:
corr

In [ ]:
sns.heatmap(corr, annot=True, xticklabels = list(rankings), yticklabels = list(rankings))

Если у нас есть данные про время, можно считать "трендовые слова": те слова, ранг которых по частотности вырос по сравнению с предыдущим периодом.

#### Задание

Возьмите две главны какой-нибудь большой книжки (или другие текстовые данные, в которых что-то известно про время создания), и найдите трендовые слова используя разницу в порядке слова по частотности.

### Графовые методы

* Вершины графа: слова
* Ребра графа могут определяться по следующим правилам:
    * Последовательные слова
    * Слова внутри левого или правого окна в $\pm$ 2-5 слов  

* Ребра могут быть взвешенные или невзвешенные, направленные или ненаправленные
* Любая мера центральности графа используется для определения важности вершин в графе. Слова, соответствующие наиболее важным вершинам, считаются ключевыми. 
* Если две соседние вершины оказываются важными, соответствующие им слова формируют ключевое словосочетание.

Меры центральностей.
![centralities](centrality_measures.png)

A) Betweenness centrality

B) Closeness centrality

C) Eigenvector centrality

D) Degree centrality

E) Harmonic centrality

F) Katz centrality of the same graph.

#### TextRank

Работает по тому же принципу, что и PageRank. Рёбра -- совстречаемость слов.

![PageRank](PageRank.jpg)

Используем TextRank для извлечения ключевых слов:

In [ ]:
!pip3 install gensim

In [ ]:
from gensim.summarization import keywords

In [ ]:
%%time
text = ' '.join(topic_texts)
kw = keywords(text)

Результаты:

In [ ]:
rankings = pd.DataFrame({'Text Rank': kw.split('\n')})
rankings.head(10)

## Мера контрастности $tf-idf$



Частота терма [Luhn, 1957]:  Важность терма в тексте пропорциональная его частоте.

Обратная документная частота [Spaerck Jones, 1972]: Специфичность терма в тексте обратно пропорциональна числу текстов, в которых терм встречается. 

$tfidf(term, text, collection) = tf(term, document) \times idf(term, collection)$

Самая популярная комбинация весов: $f_{t,d} \times \log \frac{|D|}{n_t+1}$

Извлекаем ключевые слова по $tf-idf$:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0)
tfidf_matrix =  tfidf.fit_transform([' '.join(tokens) for topic, tokens in tokens_by_topic.items()])
feature_names = tfidf.get_feature_names() 
dense = tfidf_matrix.todense()

In [ ]:
topic_id = 5

In [ ]:
text = dense[topic_id].tolist()[0]
phrase_scores = [pair for pair in zip(range(0, len(text)), text) if pair[1] > 0]
sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)

In [ ]:
tfidf_ranking = []
for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:40]:
    tfidf_ranking.append(phrase)

Результаты:

In [ ]:
rankings = pd.DataFrame({'tf-idf': tfidf_ranking})
rankings.head(10)